In [58]:
# === IMPORTS INTERNOS ===
from core.libs import pd, np
from core.db import get_engine

# === CONEXIÓN ===
engine = get_engine()

# === LISTA DE CONTRATOS ===
codes = [
    "US0166","US0167","US0168","US0169","US0170",
    "CR0112",
    "MX0108","MX0109","MX0110","MX0111","MX0112","MX0113",
    "MX0114","MX0115","MX0116","MX0117","MX0118","MX0119",
    "MX0120","MX0121","MX0122","MX0123","MX0124","MX0125",
    "MX0126","MX0127","MX0128","MX0129","MX0130","MX0131",
    "MX0132",
]




💻 Conectado a la base de datos helloworldtree


In [59]:
# === QUERY BASE (todo en minúsculas) ===
in_list = ",".join([f"'{c}'" for c in codes])
q = f"""
SELECT
    contract_code,
    status,
    etp_year
FROM masterdatabase.contract_tree_information
WHERE contract_code IN ({in_list})
"""

# === CARGA ===
df = pd.read_sql(q, engine)

# === REGION de los dos primeros caracteres (sin cambiar mayúsculas) ===
df["region"] = df["contract_code"].str[:2]

# === PIVOTEOS DINÁMICOS (sin hardcodear categorías) ===
tbl_status = (
    df.pivot_table(
        index="region",
        columns="status",
        values="contract_code",
        aggfunc="count",
        fill_value=0
    )
    .assign(total=lambda x: x.sum(axis=1))
    .reset_index()
)

# --- agregar columna con lista de contratos por región ---
contracts_by_region = (
    df.groupby("region")["contract_code"]
    .apply(lambda x: ", ".join(sorted(x.unique())))
    .reset_index(name="contract_list")
)

tbl_etp_year = (
    df.pivot_table(
        index="region",
        columns="etp_year",
        values="contract_code",
        aggfunc="count",
        fill_value=0
    )
    .assign(total=lambda x: x.sum(axis=1))
    .reset_index()
    .merge(contracts_by_region, on="region", how="left")
)



In [60]:
tbl_status.head()

status,region,Active,Pending POD,total
0,CR,1,0,1
1,MX,0,25,25
2,US,5,0,5


In [61]:
tbl_etp_year.head()

,region,2024,2025,total,contract_list
0,CR,1,0,1,CR0112
1,MX,4,21,25,"MX0108, MX0109, MX0110, MX0111, MX0112, MX0113..."
2,US,2,3,5,"US0166, US0167, US0168, US0169, US0170"


In [62]:
# === QUERY: suma + lista de contratos (con filtro IN) ===
q_trees = f"""
WITH base AS (
    SELECT DISTINCT contract_code, trees_contract, etp_year
    FROM masterdatabase.contract_tree_information
    WHERE contract_code IN ({in_list})
      AND etp_year = 2024
)
SELECT
    LEFT(contract_code, 2)                             AS region,
    SUM(trees_contract)                                AS trees_contract_2024,
    COUNT(*)                                           AS contracts_2024,
    STRING_AGG(contract_code, ', ' ORDER BY contract_code) AS contract_list_2024
FROM base
GROUP BY LEFT(contract_code, 2)
ORDER BY LEFT(contract_code, 2)
"""

trees_2024 = pd.read_sql(q_trees, engine)
trees_2024.head()


,region,trees_contract_2024,contracts_2024,contract_list_2024
0,CR,4000,1,CR0112
1,MX,17600,4,"MX0117, MX0123, MX0125, MX0126"
2,US,4391,2,"US0166, US0168"


In [63]:
# === QUERY: suma por región solo para etp_year = 2024 ===
q_trees = """
SELECT
    LEFT(contract_code, 2) AS region,
    SUM(trees_contract)    AS trees_contract_2024
FROM masterdatabase.contract_tree_information
WHERE etp_year = 2024
GROUP BY LEFT(contract_code, 2)
ORDER BY LEFT(contract_code, 2)
"""

trees_2024 = pd.read_sql(q_trees, engine)
display(trees_2024)


,region,trees_contract_2024
0,CR,4000
1,MX,46771
2,US,6591
